**Processing Covid Cases Dataset**

We first process the dataset on cases and deaths in California. We select five useful features only: date, area, cases, deaths, and total tests. 

In [ ]:
import os
import random
import pandas as pd 
import numpy as np
from datetime import datetime,timedelta
from random import randint

# from scipy import stats

df = pd.read_csv("covid19cases_test.csv")
df = df.loc[df['area'] == "California"]
df = df.astype({'date':'string'})
df = df[df['date'].notna()]
df = df[df.date >= '2020-04-22']
format = '%Y-%m-%d'
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, format))
df = df.sort_values(["date"])
df = df[['date','area', 'cases','deaths','total_tests']]
df

**Processing Covid Demographic Dataset**

We process the demogrgaphic covid dataset and sort them by date. We only select Covid related features

In [ ]:
age = pd.read_csv("covidage.csv")
age.rename(columns = {'Age Group':'age_group'}, inplace = True)
age = age.astype({'date':'string'})
age = age[age['date'].notna()]
format = '%Y-%m-%d'
age['date'] = age['date'].apply(lambda x: datetime.strptime(x, format))
age = age[age.date >= '2020-04-22']
age = age[age.age_group != 'missing']
age = age[age.age_group != 'Missing']
age = age[age.age_group != 'Total']
age = age.sort_values(["date"])
age

In [ ]:
from pandas.core.sorting import get_indexer_indexer
gender = pd.read_csv("covidgender.csv")
gender = gender.astype({'date':'string'})
gender = gender[gender['date'].notna()]
format = '%Y-%m-%d'
gender['date'] = gender['date'].apply(lambda x: datetime.strptime(x, format))
gender = gender[gender.date >= '2020-04-22']
gender = gender[gender.Gender != 'missing']
gender = gender[gender.Gender != 'Missing']
gender = gender[gender.Gender != 'Unknown']
gender = gender[gender.Gender != 'Total']
gender = gender.sort_values(["date"])
gender

In [ ]:
enthnicity = pd.read_csv("covidethnicity.csv")
enthnicity = enthnicity.astype({'date':'string'})
enthnicity = enthnicity[enthnicity['date'].notna()]
format = '%Y-%m-%d'
enthnicity['date'] = enthnicity['date'].apply(lambda x: datetime.strptime(x, format))
enthnicity = enthnicity[enthnicity.date >= '2020-04-22']
enthnicity = enthnicity[enthnicity.Ethnicity != 'missing']
enthnicity = enthnicity[enthnicity.Ethnicity != 'Missing']
enthnicity = enthnicity[enthnicity.Ethnicity != 'Unknown']
enthnicity = enthnicity[enthnicity.Ethnicity != 'Total']
enthnicity = enthnicity.sort_values(["date"])
enthnicity

In [ ]:
merged_age_gender = pd.merge(age, gender, on="date", how="left")
merged_all = pd.merge(merged_age_gender, enthnicity, on="date", how="left")
first_column = merged_all.pop('date')
  
# insert column using insert(position,column_name,
# first_column) function
merged_all.insert(0, 'date', first_column)
merged_all.replace(np.NaN, 0)
merged_all

**Merge the two datasets by date**

We merge the processed Covid cases dataset with processed demographic dataset by the date column. We're interested in daily cases, daily deaths and total tests in California on each date. At that date, we also include relevant demographic information: we incorporate percent_cases and percent_deaths of each demographic category.

In [ ]:
final=pd.merge(df, merged_all, on="date", how="left")
final = final.rename(columns={'cases': 'daily_cases'})
final = final.rename(columns={'deaths': 'daily_deaths'})
final = final.rename(columns={'total_tests': 'daily_total_tests'})
final

In [ ]:
#final.to_csv('/content/dataset.csv', encoding='utf-8', index=False)

**Due to RAM issue, the giant full dataset should be processed on server and pushed to github. We're just showing an example of exploding the first 100 rows for user-level DP**

**Finalized Dataset: Low Sensitivity of cases and daily_deaths**

To get better DP accuracy and error bounds guarantees, we explode the rows by cases and daily_deaths. For example, if we have 10 cases and 10 deaths on a row of date 2020-04-22, then we expand that row of 2020-04-22 rows by 10 x 10 rows. Then we fill in random 0's and 1's. The total count of cases and deaths from expanded rows still matches the original number in the single row. But we bring down sensitivity this way. 



In [ ]:
# final = final.head(100)
finalized_pd = final.loc[final.index.repeat(final.daily_cases.sub(final.daily_deaths) + 1)]

In [ ]:
finalized_pd['positive'] =  np.random.randint(0,2, size=len(finalized_pd))
finalized_pd['deceased'] =  np.random.randint(0,2, size=len(finalized_pd))
finalized_pd = finalized_pd.rename(columns={"age_group": "age", "Ethnicity": "ethnicity", "Gender": "gender"})
finalized_pd.drop(columns=['date', 'area', 'daily_cases', 'daily_deaths', 'daily_total_tests',
       'total_cases_by_age', 'age_based_case_rate', 'age_based_deaths',
       'age_based_death_rate', 'total_cases_by_gender',
       'gender_based_case_rate', 'gender_based_deaths',
       'gender_based_death_rate', 'total_cases_by_ethnicity',
       'ethnicity_based_case_rate', 'ethnicity_based_deaths',
       'ethnicity_based_death_rate'], inplace=True)
finalized_pd.keys()

In [ ]:
# finalized_pd_final = finalized_pd[['positive', 'deceased', 'age', 'ethnicity', 'gender']]
finalized_pd

In [ ]:
k = 2500 #100
size = 203828 #250000 
 
for i in range(k):
    df = finalized_pd[size*i:size*(i+1)]
    df.to_csv(f'blocks/covid_block_{i}.csv', index=True)

In [ ]:
import json
metadata = {}
metadata['block_size'] = size

json_object = json.dumps(metadata, indent=4)
with open("metadata.json", "w") as outfile:
    outfile.write(json_object)

